## _Track Evaluation_

- Running _`trkx_from_gnn.py`_ produces recostucted tracks (track candiates)
- Running _`trkx_reco_eval.py`_ gives reconstruction evaluation: _`eff, fake rate, dup. rate, etc`_

Let's perform inspection after track evaluation.

In [ ]:
import sys, os, glob, yaml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
sys.path.append('..')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## _A. Inspection of Epsilon in DBSCAN_

1. Various $\epsilon$ for DBSCAN in `trkx_from_gnn.py`
2. Choose the best $\epsilon$ based on _`eff, fake rate, dup. rate`_) after `trkx_reco_eval.py`

**NOTE:** The $\epsilon$ in DBSCAN is alternative to chosing a cut on the `edge_score`.

In [ ]:
# Muons: lists from eval_summary.txt
e_list = [0.10, 0.15, 0.20, 0.25, 0.35, 0.45, 0.55, 0.75, 0.85, 0.95]
eff_list = [90.86,91.68,91.68,91.58,89.84,87.58,85.83,83.57,81.52,76.28]

In [ ]:
# FWP: lists from eval_summary.txt
e_list = [0.10, 0.15, 0.20, 0.25, 0.35, 0.45, 0.55, 0.75, 0.85, 0.95]
eff_list = [92.6518,92.3720,92.0292,91.6445,90.1861,88.7871, 87.6294,85.4050,83.3345,77.8365]
fr_list = [-0.5733,-0.5928,-0.5446,-0.4382,-0.1309,0.0460, 0.1750,0.4189,0.7341,1.8218]
dr_list = [7.4034,6.8437,6.2599,5.5479,3.6662,2.6337, 2.3954,2.1506,2.0922,2.1024]

In [ ]:
# covert to ndarrays
eff = np.array(eff_list)
fr = np.array(fr_list)
dr = np.array(dr_list)
epsilon = np.array(e_list)

In [ ]:
# plt.style.use('ggplot')
# mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
# init subplots
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
ax.plot(epsilon, eff, linestyle='-', linewidth=1.5, color='blue', label="Tracking Efficiency (Phys.)")
ax.plot(epsilon, fr, linestyle='-', linewidth=1.5, color='red', label="Fake Rate")
ax.plot(epsilon, dr, linestyle='-', linewidth=1.5, color='green', label="Clone Rate")

plt.axvline(x=0.10, linewidth=0.5, color='magenta')
plt.axvline(x=0.15, linewidth=0.5, color='magenta')
plt.axvline(x=0.20, linewidth=0.5, color='magenta')
plt.axvline(x=0.25, linewidth=0.5, color='magenta')
plt.axvline(x=0.3, linewidth=0.5, color='magenta')

# plotting params
# ax.set_title('Track evaluation (efficiency vs epsilon)')
ax.set_xlabel('epsilon', fontsize=20)
ax.set_ylabel('efficiency [%]', fontsize=20)
ax.set_xlim(0, 1)
ax.set_ylim(-5, 100)
ax.grid(False)
fig.tight_layout()
# fig.savefig("epsilon_eff.png")

## _B. Visualize Test Data_

Inspect easy to difficult to reconstruct tracks. Choose 10 such events and use `score_cut=0.25`

In [ ]:
from src.drawing import detector_layout, draw_proc_event

In [ ]:
inputdir="../run_all/fwp_gnn_processed/pred"
proc_files = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
feature_data = torch.load(proc_files[0], map_location=device)

In [ ]:
feature_data

In [ ]:
event_id = int(os.path.basename(feature_data.event_file)[-4:])

In [ ]:
event_id

## _(C.) Post Analysis_

- Read evaluation data

In [ ]:
sys.path.append('..')

In [ ]:
from src.drawing import detector_layout, draw_proc_event
from src.utils_math import polar_to_cartesian

In [ ]:
# select event index (glob will fetch events as a list)
event_idx = 1

### _(1.) Read True Tracks_

In [ ]:
# get list of true tracks (test dataset)
inputdir="../run_all/fwp_gnn_processed/pred"
true_trkx = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
# event Data
true_track = torch.load(true_trkx[event_idx], map_location=device)
print("Length of Data: {}".format(len(true_track)))

In [ ]:
true_track

In [ ]:
# hit features
r, phi, ir = true_track.x.T

In [ ]:
# spatial coordinate transform
x, y = polar_to_cartesian(r.detach().numpy(), phi.detach().numpy())

In [ ]:
# compensate scaling
ir = ir.detach().numpy()*100

In [ ]:
# plot true event
fig, ax = detector_layout(figsize=(10,10))
e_id = int(true_track.event_file[-10:])
p_ids = np.unique(true_track.pid)

for pid in p_ids:
    idx = true_track.pid == pid
    ax.plot(x[idx], y[idx], '-', linewidth=1.5)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*50), label='particle_id: {}'.format(int(pid)))

ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("true_track.png")

### _(2.) Read Reco Track_

In [ ]:
inputdir="../run_all/fwp_gnn_segmenting/seg"
reco_trkx = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
print("Track Id: ", reco_trkx[event_idx])

In [ ]:
reco_track = torch.load(reco_trkx[event_idx], map_location=device)
print("Length of Data: {}".format(len(reco_track)))

In [ ]:
reco_track.head()

In [ ]:
# plot reco event, here, (x,y,ir) comes from ture event. but idx is from reco event
fig, ax = detector_layout(figsize=(10,10))
e_id = e_id
t_ids = np.unique(reco_track.track_id)

for tid in t_ids:
    idx = reco_track.track_id == tid
    ax.plot(x[idx], y[idx], '-', linewidth=2)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*100), label='particle_id: {}'.format(tid))
    
ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("reco_track.png")

### _(3.) Read Track Evaluation Data_

In [ ]:
inputdir="../run_all/fwp_gnn_segmenting/eval"

In [ ]:
evtid = int(os.path.split(reco_trkx[1006])[1])

In [ ]:
print("evtid: {}".format(evtid))

In [ ]:
!./eval_reco_evt.sh $evtid

In [ ]:
# path to HDF5 Store
hdf5_path = inputdir+"/{}_particles.h5".format(evtid)
print(hdf5_path)

In [ ]:
df = pd.read_hdf(hdf5_path)
print("Length of Data: {}".format(len(df)))

In [ ]:
df.head()

In [ ]:
# Read the Summary of Evaluation
sum_path = inputdir+"/{}_summary.txt".format(evtid)
print(sum_path)

In [ ]:
with open(sum_path) as f:
    contents = f.read()
    print(contents)